In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from gensim.models import Word2Vec
import numpy as np
import time
# Assuming 'your_data.csv' is the name of your CSV file
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
df = pd.read_csv('./data/Training-dataset.csv')
stop_words = set(stopwords.words('english'))
custom_stopwords = {'movie', 'film', 'story', 'character'}
stop_words.update(custom_stopwords)
lemmatizer = WordNetLemmatizer()
s_time = time.time()
ul_list =[]
def get_wordnet_pos(nltk_tag):

    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
for index, row in df.iterrows():
  words = word_tokenize(row['plot_synopsis'])
  Y=[]
  for word in words:
    if word.isalpha() and word.lower() not in stop_words:
      t=nltk.pos_tag([word])[0]
      wordnet_tagged = [t[0], get_wordnet_pos(t[1])]
      d=wordnet_tagged[1]
      if d !=None:
        Y.append(lemmatizer.lemmatize(word,d))
      else:
        Y.append(lemmatizer.lemmatize(word))
  ul_list.append(' '.join(Y))
df['processed_text']=ul_list

print(df['processed_text'])
vectorizer = TfidfVectorizer(ngram_range = (1,4))
X = vectorizer.fit_transform(df['processed_text'])



# Example: Calculate similarity between the first two documents

# for index, row in validation_df.iterrows():
#     term1, term2, ID = row[1], row[2], row[0]  # Added ID
#     similarity = calculate_cosine_similarity(term1, term2, training_tfidf_matrix, tfidf_vectorizer)
#     similarities_tfidf.append((ID, term1, term2, similarity))  # Include ID




# similarity_matrix = cosine_similarity(X[0], X[1])
# print(1)
# print(similarity_matrix)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0       recent amount challenge Billy Lo Bruce Lee fri...
1       city Tremont renowned investigative journalist...
2       Lankester Merrin veteran Catholic priest exorc...
3       Serendipity Seasons heartwarming romantic tell...
4       Young naive slacker Adam Jack life mum Nicky K...
                              ...                        
8252    serve eight month sentence break enter Git Hyn...
8253    Mystery crew head Chicago talent show call Tal...
8254    run Another Life revolve around Davidson famil...
8255    North Bend Psychiatric Hospital young patient ...
8256    depiction various scene usually violent bizarr...
Name: processed_text, Length: 8257, dtype: object


In [4]:
feature_names =vectorizer.get_feature_names_out()
print(feature_names)
print(len(feature_names))

['aa' 'aa another' 'aa another storyline' ... '齐天大圣 traditional'
 '齐天大圣 traditional chinese' '齐天大圣 traditional chinese 齊天大聖']
9351241


In [5]:
validation_dataset_path = './data/Task-1-validation-dataset.csv'
validation_df = pd.read_csv(validation_dataset_path, header=None)


# Calculating cosine similarity for each term pair using TF-IDF document embeddings
similarities_tfidf = []

for index, row in validation_df.iterrows():
    term1, term2, ID = row[1], row[2], row[0]  # Added ID
    idx_term1 = vectorizer.vocabulary_.get(term1)
    idx_term2 = vectorizer.vocabulary_.get(term2)
    # If both terms are in the vocabulary, calculating cosine similarity
    if idx_term1 is not None and idx_term2 is not None:
        # Extracting the TF-IDF vectors for the two terms
        vector1 = X.getcol(idx_term1)
        vector2 = X.getcol(idx_term2)
        similarity = cosine_similarity(vector1.T, vector2.T)[0][0]
    else:
        # If either term is not in the vocabulary, return 0
        similarity = 0
    similarities_tfidf.append((ID, term1, term2, similarity))  # Include ID

# Displaying the calculated cosine similarities for validation
print("ID        Term Pair               Calculated Cosine Similarity (TF-IDF)")
for ID, term1, term2, similarity in similarities_tfidf:
    print(f"{ID}\t{term1} - {term2}:\t\t{similarity}")

# Creating a DataFrame from the results
r = pd.DataFrame(similarities_tfidf, columns=['ID', 'Term1', 'Term2', 'Cosine_Similarity_TFIDF'])
sc = r[['ID', 'Cosine_Similarity_TFIDF']]

# Saving the results to a CSV file
sc.to_csv('./data/10949167-Task1-method-a-validation.csv', header=False,index=False)

ID        Term Pair               Calculated Cosine Similarity (TF-IDF)
1	absorb - learn:		0.03975916237754672
2	absorb - withdraw:		0.0
3	achieve - accomplish:		0.04976176621932748
4	achieve - try:		0.05691392393696167
6	acquire - get:		0.05592279652347755
7	acquire - obtain:		0.07356886708606798
8	acquire - find:		0.09631877082396961
11	apple - sauce:		0.0
12	apple - lemon:		0.00039756174756963233
13	apple - sunshine:		0.0
14	area - region:		0.102968695691692
16	area - corner:		0.08951759046097657
17	arm - shoulder:		0.11782208862612739
20	arm - body:		0.21882061896026128
21	arm - vein:		0.022450976533636157
22	arm - knee:		0.052790843376129416
23	arm - bone:		0.059185272444194297
24	arm - neck:		0.1546765762774087
25	ball - cannon:		0.013651780094409425
26	ball - basket:		0.019637331817808336
29	bath - trick:		0.03410129959082843
30	bath - wife:		0.05350760458212363
31	bath - balloon:		0.01138864653533202
32	bed - bedroom:		0.24236929695083104
33	bed - blanket:		0.07195130425181709


In [6]:
#!python task1_eval_script_student_version.py 10949167-Task1-method-a-validation.csv Task-1-validation-dataset.csv

The following simalarity scores may need checking:
(achieve,accomplish) similarity score: 0.04976176621932748, gold ranking: 8.57
(achieve,try) similarity score: 0.05691392393696167, gold ranking: 4.42
----------------------------
(acquire,get) similarity score: 0.05592279652347755, gold ranking: 8.82
(acquire,obtain) similarity score: 0.07356886708606798, gold ranking: 8.57
----------------------------
(acquire,get) similarity score: 0.05592279652347755, gold ranking: 8.82
(acquire,find) similarity score: 0.09631877082396961, gold ranking: 6.38
----------------------------
(acquire,obtain) similarity score: 0.07356886708606798, gold ranking: 8.57
(acquire,find) similarity score: 0.09631877082396961, gold ranking: 6.38
----------------------------
(apple,sauce) similarity score: 0.0, gold ranking: 1.43
(apple,sunshine) similarity score: 0.0, gold ranking: 0.58
----------------------------
(arm,shoulder) similarity score: 0.11782208862612739, gold ranking: 4.85
(arm,body) similarity sco

In [7]:
validation_dataset_path = './data/Task-1-test-dataset2.csv'
validation_df = pd.read_csv(validation_dataset_path, header=None)

def calculate_cosine_similarity(term1, term2, X, vectorizer):

    idx_term1 = vectorizer.vocabulary_.get(term1)
    idx_term2 = vectorizer.vocabulary_.get(term2)
    # If both terms are in the vocabulary, calculating cosine similarity
    if idx_term1 is not None and idx_term2 is not None:
        # Extracting the TF-IDF vectors for the two terms
        vector1 = X.getcol(idx_term1)
        vector2 = X.getcol(idx_term2)
        similarity = cosine_similarity(vector1.T, vector2.T)[0][0]
    else:
        # If either term is not in the vocabulary, return 0
        similarity = 0

    return similarity


# Calculating cosine similarity for each term pair using TF-IDF document embeddings
similarities_tfidf = []

for index, row in validation_df.iterrows():
    term1, term2, ID = row[1], row[2], row[0]  # Added ID
    similarity = calculate_cosine_similarity(term1, term2, X, vectorizer)
    similarities_tfidf.append((ID, term1, term2, similarity))  # Include ID

# Displaying the calculated cosine similarities for validation
print("ID        Term Pair               Calculated Cosine Similarity (TF-IDF)")
for ID, term1, term2, similarity in similarities_tfidf:
    print(f"{ID}\t{term1} - {term2}:\t\t{similarity}")

# Creating a DataFrame from the results
r = pd.DataFrame(similarities_tfidf, columns=['ID', 'Term1', 'Term2', 'Cosine_Similarity_TFIDF'])
sc = r[['ID', 'Cosine_Similarity_TFIDF']]

# Saving the results to a CSV file
sc.to_csv('./data/10949167-Task1-method-a.csv', header=False,index=False)

ID        Term Pair               Calculated Cosine Similarity (TF-IDF)
12	bad - awful:		0.04079614496408917
54	bad - terrific:		0.004454892998607996
332	beach - island:		0.1647836641932744
422	beach - reef:		0.05350641017260341
557	car - bridge:		0.11659208908500238
499	car - carriage:		0.04800338468311045
256	door - gate:		0.08239541278351138
568	door - kitchen:		0.25116252147521273
20	hard - tough:		0.037508345995145964
15	hard - simple:		0.03052218155165635
438	house - carpet:		0.060495634827011785
555	house - barn:		0.12401302674868454
2028	journey - voyage:		0.030895977627540164
440	journey - conquest:		0.010713806773870228
908	keep - protect:		0.12271754913619319
959	keep - give:		0.3551045901417291
43	large - big:		0.10070651436934347
110	large - flexible:		0.016207642503072262
793	leave - enter:		0.2680324727731846
823	leave - go:		0.5495077730266953
733	task - highway:		0.015151074622892577
542	task - job:		0.05613930228263561
35	weird - odd:		0.009390071322593544
34	weird - 

In [8]:
e_time = time.time()
print(e_time-s_time)

835.3973188400269


In [9]:
s_time=time.time()
ul2=[]
for d in ul_list:
  ul2.append(d.split())


In [10]:
df['processed_text']=ul2

In [11]:
word2vec_model = Word2Vec(sentences=df['processed_text'], vector_size=300, window=30, min_count=1, workers=4)

In [12]:
df['dense_representation'] = df['plot_synopsis'].apply(lambda x: np.mean([word2vec_model.wv[word] for word in word_tokenize(x.lower()) if word in word2vec_model.wv], axis=0))

In [13]:
validation_dataset_path = './data/Task-1-validation-dataset.csv'
validation_df = pd.read_csv(validation_dataset_path, header=None)
def calculate_cosine_similarity(term1, term2, model):
    vectors1 = [model.wv[word] for word in word_tokenize(term1.lower()) if word in model.wv]
    vectors2 = [model.wv[word] for word in word_tokenize(term2.lower()) if word in model.wv]

    # Check if vectors are empty (no words in the vocabulary)
    if not vectors1 or not vectors2:
        return 0

    # Calculate mean vectors
    vector1 = np.mean(vectors1, axis=0, keepdims=True)
    vector2 = np.mean(vectors2, axis=0, keepdims=True)

    similarity = cosine_similarity(vector1, vector2)
    return similarity[0][0]
# Calculate cosine similarity for each term pair using Word2Vec embeddings
similarities_word2vec = []

for index, row in validation_df.iterrows():
    term1, term2, ID = row[1], row[2], row[0]

    # Calculate similarity for the current term pair
    similarity = calculate_cosine_similarity(term1, term2, word2vec_model)
    similarities_word2vec.append((ID, term1, term2, similarity))  # Include ID

# Displaying the calculated cosine similarities for validation
print("ID        Term Pair               Calculated Cosine Similarity (TF-IDF)")
for ID, term1, term2, similarity in similarities_word2vec:
    print(f"{ID}\t{term1} - {term2}:\t\t{similarity}")

# Creating a DataFrame from the results
r = pd.DataFrame(similarities_word2vec, columns=['ID', 'Term1', 'Term2', 'Cosine_Similarity_TFIDF'])
sc = r[['ID', 'Cosine_Similarity_TFIDF']]

# Saving the results to a CSV file
sc.to_csv('./data/10949167-Task1-method-b-validation.csv', header=False, index=False)

ID        Term Pair               Calculated Cosine Similarity (TF-IDF)
1	absorb - learn:		0.20129111409187317
2	absorb - withdraw:		0.3136787414550781
3	achieve - accomplish:		0.8770895004272461
4	achieve - try:		-0.06291495263576508
6	acquire - get:		-0.15586601197719574
7	acquire - obtain:		0.811968982219696
8	acquire - find:		-0.018530547618865967
11	apple - sauce:		0.6777719259262085
12	apple - lemon:		0.465087890625
13	apple - sunshine:		0.5377141833305359
14	area - region:		0.4458329975605011
16	area - corner:		0.2049059420824051
17	arm - shoulder:		0.6682100892066956
20	arm - body:		0.27202972769737244
21	arm - vein:		0.24850349128246307
22	arm - knee:		0.5911325216293335
23	arm - bone:		0.44607898592948914
24	arm - neck:		0.6497403383255005
25	ball - cannon:		0.39040401577949524
26	ball - basket:		0.5140148997306824
29	bath - trick:		0.22183838486671448
30	bath - wife:		0.10199359804391861
31	bath - balloon:		0.4372178316116333
32	bed - bedroom:		0.7608644962310791
33	bed - bl

In [14]:
#!python task1_eval_script_student_version.py 10949167-Task1-method-b-validation.csv Task-1-validation-dataset.csv

The following simalarity scores may need checking:
(absorb,learn) similarity score: 0.20129111409187317, gold ranking: 5.48
(absorb,withdraw) similarity score: 0.3136787414550781, gold ranking: 2.97
----------------------------
(acquire,get) similarity score: -0.15586601197719574, gold ranking: 8.82
(acquire,obtain) similarity score: 0.811968982219696, gold ranking: 8.57
----------------------------
(acquire,get) similarity score: -0.15586601197719574, gold ranking: 8.82
(acquire,find) similarity score: -0.018530547618865967, gold ranking: 6.38
----------------------------
(apple,sauce) similarity score: 0.6777719259262085, gold ranking: 1.43
(apple,lemon) similarity score: 0.465087890625, gold ranking: 4.05
----------------------------
(apple,lemon) similarity score: 0.465087890625, gold ranking: 4.05
(apple,sunshine) similarity score: 0.5377141833305359, gold ranking: 0.58
----------------------------
(arm,body) similarity score: 0.27202972769737244, gold ranking: 4.05
(arm,knee) sim

In [15]:
validation_dataset_path = './data/Task-1-test-dataset2.csv'
validation_df = pd.read_csv(validation_dataset_path, header=None)
def calculate_cosine_similarity(term1, term2, model):
    vectors1 = [model.wv[word] for word in word_tokenize(term1.lower()) if word in model.wv]
    vectors2 = [model.wv[word] for word in word_tokenize(term2.lower()) if word in model.wv]

    # Check if vectors are empty (no words in the vocabulary)
    if not vectors1 or not vectors2:
        return 0

    # Calculate mean vectors
    vector1 = np.mean(vectors1, axis=0, keepdims=True)
    vector2 = np.mean(vectors2, axis=0, keepdims=True)

    similarity = cosine_similarity(vector1, vector2)
    return similarity[0][0]
# Calculate cosine similarity for each term pair using Word2Vec embeddings
similarities_word2vec = []

for index, row in validation_df.iterrows():
    term1, term2, ID = row[1], row[2], row[0]

    # Calculate similarity for the current term pair
    similarity = calculate_cosine_similarity(term1, term2, word2vec_model)
    similarities_word2vec.append((ID, term1, term2, similarity))  # Include ID

# Displaying the calculated cosine similarities for validation
print("ID        Term Pair               Calculated Cosine Similarity (TF-IDF)")
for ID, term1, term2, similarity in similarities_word2vec:
    print(f"{ID}\t{term1} - {term2}:\t\t{similarity}")

# Creating a DataFrame from the results
r = pd.DataFrame(similarities_word2vec, columns=['ID', 'Term1', 'Term2', 'Cosine_Similarity_TFIDF'])
sc = r[['ID', 'Cosine_Similarity_TFIDF']]

# Saving the results to a CSV file
sc.to_csv('./data/10949167-Task1-method-b.csv', header=False, index=False)

ID        Term Pair               Calculated Cosine Similarity (TF-IDF)
12	bad - awful:		0.4839628040790558
54	bad - terrific:		0.09407765418291092
332	beach - island:		0.5475234985351562
422	beach - reef:		0.6172888875007629
557	car - bridge:		0.3698665499687195
499	car - carriage:		0.22711677849292755
256	door - gate:		0.5824406147003174
568	door - kitchen:		0.5730670094490051
20	hard - tough:		0.42964914441108704
15	hard - simple:		0.42558470368385315
438	house - carpet:		0.15773726999759674
555	house - barn:		0.595558226108551
2028	journey - voyage:		0.44800740480422974
440	journey - conquest:		0.49094751477241516
908	keep - protect:		0.263264000415802
959	keep - give:		0.24984581768512726
43	large - big:		0.2820632755756378
110	large - flexible:		0.3735213577747345
793	leave - enter:		0.32072827219963074
823	leave - go:		0.4118373990058899
733	task - highway:		-0.3004300892353058
542	task - job:		0.18079741299152374
35	weird - odd:		0.6963600516319275
34	weird - normal:		0.3736103

In [16]:
e_time=time.time()
print(e_time-s_time)

182.79399609565735
